# GÖREV 1: Elasticsearch Kurulumu
- Docker servisleri çalışıyor mu kontrol ediniz.
- Elasticsearch'i kurununuz ve çalıştığını kontol ediniz. 
- Mozilla üzerinden kibana sayfasına giriş yapınız.

In [ ]:
# Docker servisleri çalışıyor mu kontrol edininiz.

# sudo system status docker
# sudo systemctl start docker

In [ ]:
# Elasticsearch'i kurununuz ve çalıtığını kontol ediniz. 
# docker-compose -f elk-docker-compose.yml up -d

In [ ]:
# Bütün kontainerlar kurulmuş mu listeleyiniz.
# docker-compose -f elk-docker-compose.yml ps

In [ ]:
# mozilla üzerinden kibana sayfasına giriş yapınız.
# localhost:5601

In [ ]:
# venvspark aktif edeceğiz
# source venvspark/bin/activate

In [ ]:
# jupyter notebook 

# GÖREV 2: Veriyi Anlama (Data Understanding) ve Hazırlama
- Gerekli Kütüphaneleri yükleyiniz.
- Veriyi indiriniz.
- Veriyi okuyununuz.
- ilk 5 gözlemini 
- toplam gözlem sayısı
- toplam değişken sayısı
- değişken tiplerini inceleyiniz.

In [ ]:
pip install elasticsearch==7.14.0

You should consider upgrading via the '/home/train/venvspark/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers

In [ ]:
es = Elasticsearch("localhost:9200") 

In [ ]:
! wget https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv

--2022-10-14 21:18:44--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174240 (170K) [text/plain]
Saving to: ‘products.csv.4’

100%[======================================>] 174,240     --.-K/s   in 0.1s    

2022-10-14 21:18:45 (1.18 MB/s) - ‘products.csv.4’ saved [174240/174240]



In [ ]:
!ls -l

total 916
-rwxrw-rw-. 1 train train   1313 Jul 17 14:00 elk-docker-compose.yml
-rw-rw-r--. 1 train train 174240 Jul  7 16:36 products.csv
-rw-rw-r--. 1 train train 174240 Oct  7 13:51 products.csv.1
-rw-rw-r--. 1 train train 174240 Oct  9 18:02 products.csv.2
-rw-rw-r--. 1 train train 174240 Oct 13 16:24 products.csv.3
-rw-rw-r--. 1 train train 174240 Oct 14 21:18 products.csv.4
-rw-rw-r--. 1 train train  23698 Oct 14 20:51 proje_elk-atolye.ipynb
-rwxrw-rw-. 1 train train  25587 Oct  9 18:30 proje_elk.ipynb


In [ ]:
df = pd.read_csv("file:///home/train/elasticsearch/products.csv")

In [ ]:
df.head()

,productId,productCategoryId,productName,productDescription,productPrice,productImage
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,NaN,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,Under Armour Men's Highlight MC Football Clea,NaN,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,Under Armour Men's Renegade D Mid Football Cl,NaN,89.99,http://images.acmesports.sports/Under+Armour+M...
3,4,2,Under Armour Men's Renegade D Mid Football Cl,NaN,89.99,http://images.acmesports.sports/Under+Armour+M...
4,5,2,Riddell Youth Revolution Speed Custom Footbal,NaN,199.99,http://images.acmesports.sports/Riddell+Youth+...


##  Veriyi Anlama - Eksik gözlemler

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   productId           1345 non-null   int64  
 1   productCategoryId   1345 non-null   int64  
 2   productName         1345 non-null   object 
 3   productDescription  0 non-null      float64
 4   productPrice        1345 non-null   float64
 5   productImage        1345 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 63.2+ KB


In [ ]:
df.drop('productDescription', inplace=True, axis=1) 

In [ ]:
df.head()

,productId,productCategoryId,productName,productPrice,productImage
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,Under Armour Men's Highlight MC Football Clea,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,Under Armour Men's Renegade D Mid Football Cl,89.99,http://images.acmesports.sports/Under+Armour+M...
3,4,2,Under Armour Men's Renegade D Mid Football Cl,89.99,http://images.acmesports.sports/Under+Armour+M...
4,5,2,Riddell Youth Revolution Speed Custom Footbal,199.99,http://images.acmesports.sports/Riddell+Youth+...


# GÖREV 3: Veriyi Elasticsearch'e gönderme
- Veri seti için mapping hazırlayınız. Elasticsearch üzerinde indices oluşturunuz.
- Veriyi Elasticsearch'e gönderiniz. 
- productName içeirisnde Football geçen ürünleri listeleyiniz.

In [ ]:
products_index =  {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer":
          {
            "type":"custom",
            "tokenizer":"standard",
            "filter":[
              "lowercase", "custom_edge_ngram","asciifolding"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram":2,
            "max_gram": 10
            }
          }
        }
      }
    },
    "mappings": {
    "properties": {
      "productId":  { "type": "keyword"  }, 
      "productName": { "type": "text"  },
      "productPrice": {"type": "integer"},
      "productPrice": {"type": "float"},
      "productImage": {"type": "keyword"}
    }
  }
  }

In [ ]:
df.columns 

Index(['productId', 'productCategoryId', 'productName', 'productPrice',
       'productImage'],
      dtype='object')

In [ ]:
try:
    es.indices.delete("products_python")
except:
    print("No index")
    

In [ ]:
es.indices.create("products_python", body=products_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products_python'}

In [ ]:
def df_to_es(df, es_index):
    for df_ix, line in df.iterrows():
        
        yield {
            "_index": es_index,
            "_id": df_ix,
            "_source": {
                'productId': line[0], 
                'productCategoryId': line[1], 
                'productName': line[2], 
                'productPrice': line[3], 
                'productImage': line[4]
                }
            }

In [ ]:
helpers.bulk(es, df_to_es(df, "products_python"), raise_on_error=False)

(1345, [])

In [ ]:
keyword = "Football"
res = es.search(index='products_python', body={
    "query": {
        "bool": {
            "should": [
                {
                    "match": {
                        "productName": keyword
                    }
                }
            ]
        }
    }
    
})

In [ ]:
res['hits']['max_score'] 

3.1810076

In [ ]:
res['hits']['hits'][:2]   

[{'_index': 'products_python',
  '_type': '_doc',
  '_id': '618',
  '_score': 3.1810076,
  '_source': {'productId': 619,
   'productCategoryId': 41,
   'productName': 'Nike Vapor Crew Football Sock',
   'productPrice': 18.0,
   'productImage': 'http://images.acmesports.sports/Nike+Vapor+Crew+Football+Sock'}},
 {'_index': 'products_python',
  '_type': '_doc',
  '_id': '1143',
  '_score': 3.1810076,
  '_source': {'productId': 1144,
   'productCategoryId': 51,
   'productName': 'Wilson NFL Autograph Official Football',
   'productPrice': 44.99,
   'productImage': 'http://images.acmesports.sports/Wilson+NFL+Autograph+Official+Football'}}]

# GÖREV 4: KİBANA

- Bütün indices'ları listeleyininiz.
- Kibana konsolu üzerinden produtName'de Football içeren dökümanlari listeleyiniz.
- Elasticsearh'e gönderidiğiniz veri setinin ilk 10 gözlemini kibana üzerinden sql query'si ile gözlemleyiniz.

In [ ]:
"""
GET /_cat/indices?v
"""

In [ ]:
"""
GET /products_python/_search
{
  "query": {
    "match": {
      "productName": "Football"
    }
  }
}
"""

In [ ]:
"""
POST _sql?format=txt
{
  "query": """
  SELECT * FROM "products_python" LIMIT 10
  """ 
}
"""